In [4]:
#dependencies
import requests
import os
import pandas as pd
import statistics
from pandas import Series
from matplotlib import pyplot as plt
from statsmodels.tsa.ar_model import AR
from sklearn.metrics import mean_squared_error
import numpy
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.ar_model import ARResults

  

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
kind = 'CO'
state =  "WV"
choice = "../Data/pollution.csv"

In [ ]:
choice_df = pd.read_csv(choice)

In [2]:
choice_df

NameError: name 'choice_df' is not defined

In [ ]:
index = choice_df.Year.unique()
lst = choice_df[choice_df['State']==state][kind].values

In [ ]:
series = pd.Series(lst, index = index)

In [ ]:
print(series.head())
series.plot()
plt.show()

In [ ]:

 
# create a difference transform of the dataset
def difference(dataset):
	diff = list()
	for i in range(1, len(dataset)):
		value = dataset[i] - dataset[i - 1]
		diff.append(value)
	return numpy.array(diff)
 
# Make a prediction give regression coefficients and lag obs
def predict(coef, history):
	yhat = coef[0]
	for i in range(1, len(coef)):
		yhat += coef[i] * history[-i]
	return yhat
 

# split dataset
X = difference(series.values)
size = int(len(X) * 0.80)
train, test = X[0:size], X[size:]
# train autoregression
model = AR(train)
model_fit = model.fit(maxlag=3, disp=True)
window = model_fit.k_ar
coef = model_fit.params
# walk forward over time steps in test
history = [train[i] for i in range(len(train))]
predictions = list()
for t in range(len(test)):
	yhat = predict(coef, history)
	obs = test[t]
	predictions.append(yhat)
	history.append(obs)
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
# plot
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()

In [ ]:
 
# create a difference transform of the dataset
def difference(dataset):
	diff = list()
	for i in range(1, len(dataset)):
		value = dataset[i] - dataset[i - 1]
		diff.append(value)
	return numpy.array(diff)
 
# load dataset
X = difference(series.values)
# fit model
model = AR(X)
model_fit = model.fit(maxlag=3, disp=False)
# save model to file
model_fit.save('../Model_Info/ar_model.pkl')
# save the differenced dataset
numpy.save('../Model_Info/ar_data.npy', X)
# save the last ob
numpy.save('../Model_Info/ar_obs.npy', [series.values[-1]])

In [ ]:
# load the AR model from file
loaded = ARResults.load('../Model_Info/ar_model.pkl')
print(loaded.params)
data = numpy.load('../Model_Info/ar_data.npy')
print(data)
last_ob = numpy.load('../Model_Info/ar_obs.npy')
print(last_ob)

In [ ]:
# load dataset
X = difference(series.values)
# fit model
window_size = 3
model = AR(X)
model_fit = model.fit(maxlag=window_size, disp=False)
# save coefficients
coef = model_fit.params
numpy.save('../Model_Info/man_model.npy', coef)
# save lag
lag = X[-window_size:]
numpy.save('../Model_Info/man_data.npy', lag)
# save the last ob
numpy.save('../Model_Info/man_obs.npy', [series.values[-1]])

In [ ]:

coef = numpy.load('../Model_Info/man_model.npy')
lag = numpy.load('../Model_Info/man_data.npy')
last_ob = numpy.load('../Model_Info/man_obs.npy')


In [ ]:
prediction_X = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
yhat = []
for prediction in prediction_X:
    import numpy
    def predict(coef, history):
        yhat = coef[0]
        for i in range(1, len(coef)):
            yhat += coef[i] * history[-i]
        return yhat

    # load model
    coef = numpy.load('../Model_Info/man_model.npy')
    lag = numpy.load('../Model_Info/man_data.npy')
    last_ob = numpy.load('../Model_Info/man_obs.npy')
    # make prediction
    prediction = predict(coef, lag)
    # transform prediction
    y_predict = prediction + last_ob[0]
    yhat.append(y_predict)
    # get real observation
    observation = prediction
    # update and save differenced observation
    lag = numpy.load('../Model_Info/man_data.npy')
    last_ob = numpy.load('../Model_Info/man_obs.npy')
    diffed = observation - last_ob[0]
    lag = numpy.append(lag[1:], [diffed], axis=0)
    numpy.save('../Model_Info/man_data.npy', lag)
    # update and save real observation
    last_ob[0] = observation
    numpy.save('../Model_Info/man_obs.npy', last_ob)

print(yhat)    